# Housing Case Study

## Introduction

A US-based housing company named Surprise Housing has decided to enter the Australian market. 
The company uses data analytics to purchase houses at a price below their actual values 
and flip them on at a higher price.

The company is looking at prospective properties to buy to enter the market. 
We are required to build a regression model using regularisation 
in order to predict the actual value of the prospective properties and decide whether to invest in them or not.

## Goals

1. Which variables are significant in predicting the price of a house

2. How well those variables describe the price of a house.

3. Also, determine the optimal value of lambda for ridge and lasso regression.

In [ ]:
#Import Python libraries

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import seaborn as sns
sns.set_palette('muted')
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
import os
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
# set the max columns to none to see all the columns & rows in dataset

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Import the dataset

In [ ]:
house_original = pd.read_csv('train.csv')
house_original.head()

### Checking Data details in the dataset

In [ ]:
house_original.shape

In [ ]:
house_original.info()

In [ ]:
house_original.describe()

In [ ]:
# Check for null values

house_original.isnull().sum()

In [ ]:
## Creating a copy of original dataset for analysis

house = house_original.copy()
house.head()

In [ ]:
## Verify the null percentage

house_null_percentage = (house.isnull().sum() / house.isnull().count()*100).sort_values(ascending = False)
house_null_percentage.head(15)

In [ ]:
# Columns with null values more than 40% should be dropped

house.drop(['PoolQC','MiscFeature','Alley','Fence','MasVnrType','FireplaceQu'],axis=1,inplace=True)

In [ ]:
# Fixing missing values in column LotFrontage, MasVnrArea and GarageYrBlt by imputing data

house['LotFrontage']= house.LotFrontage.fillna(house.LotFrontage.median())
house['MasVnrArea']= house.MasVnrArea.fillna(house.MasVnrArea.median())   
house['GarageYrBlt']= house.GarageYrBlt.fillna(house.GarageYrBlt.mean())



In [ ]:
## Verify the null percentage again after columns removal and imputation

house_null_percentage = (house.isnull().sum() / house.isnull().count()*100).sort_values(ascending = False)
house_null_percentage.head(15)

In [ ]:
#Replacing null with No Garrage for better understanding when house doesn't have the garage

house['GarageType'] = house['GarageType'].replace(np.nan, 'No Garage')
house['GarageFinish'] = house['GarageFinish'].replace(np.nan, 'No Garage')
house['GarageCond'] = house['GarageCond'].replace(np.nan, 'No Garage')
house['GarageQual'] = house['GarageQual'].replace(np.nan, 'No Garage')
#surp['MasVnrType'] = surp['MasVnrType'].replace(np.nan, 'None') # replacing nan with the top option of this field
house['Electrical'] = house['Electrical'].replace(np.nan, 'SBrkr') # replacing nan with the top option of this field



In [ ]:
#Changing num to categorical so for these dummy variables can be created

house['MSSubClass']=house['MSSubClass'].replace({20:'1-STORY 1946 & NEWER ALL STYLES',30:'1-STORY 1945 & OLDER',40:'1-STORY W/FINISHED ATTIC ALL AGES',
        45:'1-1/2 STORY - UNFINISHED ALL AGES',
        50:'1-1/2 STORY FINISHED ALL AGES',
        60:'2-STORY 1946 & NEWER',
        70:'2-STORY 1945 & OLDER',
        75:'2-1/2 STORY ALL AGES',
        80:'SPLIT OR MULTI-LEVEL',
        85:'SPLIT FOYER',
        90:'DUPLEX - ALL STYLES AND AGES',
       120:'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
       150:'1-1/2 STORY PUD - ALL AGES',
       160:'2-STORY PUD - 1946 & NEWER',
       180:'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
       190:'2 FAMILY CONVERSION - ALL STYLES AND AGES'})

house['OverallQual']=house['OverallQual'].replace({ 10:'Very Excellent',
       9:'Excellent',
       8:'Very Good',
       7:'Good',
       6:'Above Average',
       5:'Average',
       4:'Below Average',
       3:'Fair',
       2:'Poor',
       1:'Very Poor'})
house['OverallCond']=house['OverallCond'].replace({ 10:'Very Excellent',
       9:'Excellent',
       8:'Very Good',
       7:'Good',
       6:'Above Average',
       5:'Average',
       4:'Below Average',
       3:'Fair',
       2:'Poor',
       1:'Very Poor'})

In [ ]:
house.head()

In [ ]:
#Replacing null with No Basement when house doesn't have the basement

house['BsmtExposure'] = house['BsmtExposure'].replace(np.nan, 'No Basement')
house['BsmtFinType2'] = house['BsmtFinType2'].replace(np.nan, 'No Basement')
house['BsmtFinType1'] = house['BsmtFinType1'].replace(np.nan, 'No Basement')
house['BsmtCond'] = house['BsmtCond'].replace(np.nan, 'No Basement')
house['BsmtQual'] = house['BsmtQual'].replace(np.nan, 'No Basement')

In [ ]:
# All the missing values have been fixed

missing_values = (house.isnull().sum() / house.isnull().count()*100).sort_values(ascending = False)
missing_values.head(10)

In [ ]:
#Columns to be be categorised into numerical and categorical after checking for unique values

house.nunique()

In [ ]:
# Check for SalePrice which seems to be skewed

house['SalePrice'].describe()
sns.distplot(house['SalePrice'])

In [ ]:
# Value of skewness

print("Skewness: %f" % house['SalePrice'].skew())

In [ ]:
#transforming to have Normal disribution

house['SalePrice']=np.log(house.SalePrice) 
house['SalePrice'].describe()

In [ ]:
# Completed Normally distributed

sns.distplot(house['SalePrice']) 

# Value of skewness 

print("Skewness: %f" % house['SalePrice'].skew())

# Exploratory Data Analysis

In [ ]:
## Sale Price variation due to categorical variables

cat_cols = house.select_dtypes(include=['object'])
cat_cols.columns

In [ ]:
#Visualising the countplot and the effect of that categorical variables on dependent variable i.e. SalePrice

plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'KitchenQual',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'Functional', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'OverallQual', data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,4)

sns.countplot(x = 'OverallCond', data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,5)

sns.countplot(x = 'MSSubClass',  data = house)
plt.xticks(rotation=90)

plt.show()

In [ ]:
#Checking other columns for skewness

house['MSSubClass'].value_counts() 

In [ ]:
# Dropping columns which are highly skewed 

house.drop(['Functional'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,2,1)

sns.boxplot(x ='KitchenQual', y ='SalePrice', data = house)
plt.subplot(2,2,2)

sns.boxplot(x = 'OverallQual', y = 'SalePrice', data = house)
plt.xticks(rotation=90)
plt.subplot(2,2,3)

sns.boxplot(x = 'OverallCond', y = 'SalePrice', data = house)
plt.xticks(rotation=90)
plt.subplot(2,2,4)

sns.boxplot(x = 'MSSubClass', y = 'SalePrice', data = house)
plt.xticks(rotation=90)

plt.show()

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'GarageType',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'GarageFinish', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'GarageQual', data = house)
plt.subplot(2,3,4)

sns.countplot(x = 'GarageCond', data = house)
plt.subplot(2,3,5)

sns.countplot(x = 'SaleType',  data = house)
plt.subplot(2,3,6)

sns.countplot(x = 'SaleCondition', data = house)

plt.show()

In [ ]:
#Checking other columns for skewness

house['SaleCondition'].value_counts() 

In [ ]:
house['GarageType'] = house['GarageType'].replace(['Basment','CarPort','2Types'],'Others')
house['SaleCondition'] = house['SaleCondition'].replace(['Family','Alloca','AdjLand'],'Others')

# Dropping highly skewed column 
house.drop(['GarageQual','GarageCond','SaleType'],axis=1,inplace=True)

In [ ]:
# Variation of SalePrice on Garage type and GarageFinish 

plt.figure(figsize=(20, 12))
plt.subplot(1,2,1)

sns.boxplot(x = 'GarageType', y ='SalePrice', data = house)
plt.subplot(1,2,2)

sns.boxplot(x = 'GarageFinish', y = 'SalePrice', data = house)

## Price of Builtin Garagetype and Finished garage is the highest from below plots

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'ExterQual',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'ExterCond', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'Foundation', data = house)
plt.subplot(2,3,4)

sns.countplot(x = 'Heating', data = house)
plt.subplot(2,3,5)

sns.countplot(x = 'HeatingQC',  data = house)
plt.subplot(2,3,6)

sns.countplot(x = 'CentralAir', data = house)

plt.show()

In [ ]:
#Checking for all variables to check the skewness

house['HeatingQC'].value_counts() 

house['Foundation'] = house['Foundation'].replace(['Slab','Stone','Wood'],'Others')

#Dropping columns which are highly skewed 
house.drop(['CentralAir','Heating','ExterCond'],axis=1,inplace=True)

In [ ]:
#Impact of Garage type and GarageFinish on SalePrice

plt.figure(figsize=(20, 12))
plt.subplot(1,3,1)

sns.boxplot(x = 'ExterQual', y ='SalePrice', data = house)
plt.subplot(1,3,2)

sns.boxplot(x = 'Foundation', y = 'SalePrice', data = house)
plt.subplot(1,3,3)

sns.boxplot(x = 'HeatingQC', y = 'SalePrice', data = house)

In [ ]:
# 1. Price of Excellent ExterQual and HeatingQc is highest
# 2. Price of Poured Contrete Foundation is highest.

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'HouseStyle',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'RoofStyle', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'RoofMatl', data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,4)

sns.countplot(x = 'Exterior1st', data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,5)

sns.countplot(x = 'Exterior2nd',  data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,6)

plt.show()

In [ ]:
#Checking the skewness for other columns
house['Exterior2nd'].value_counts() 

house['HouseStyle'] = house['HouseStyle'].replace(['SFoyer','1.5Unf','2.5Unf','2.5Fin'],'Others')
house['RoofStyle'] = house['RoofStyle'].replace(['Shed','Mansard','Gambrel','Flat'],'Others')
house['Exterior1st'] = house['Exterior1st'].replace(['AsphShn','ImStucc','CBlock','Stone','BrkComm','AsbShng','Stucco','WdShing'],'Others')
house['Exterior2nd'] = house['Exterior2nd'].replace(['Other','AsphShn','ImStucc','CBlock','Stone','BrkComm','AsbShng','Stucco','BrkFace'],'Others')

#Dropping columns which are highly skewed
house.drop(['RoofMatl'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.boxplot(x = 'HouseStyle', y ='SalePrice', data = house)
plt.subplot(2,3,2)

sns.boxplot(x = 'RoofStyle', y = 'SalePrice', data = house)
plt.subplot(2,3,3)

sns.boxplot(x = 'Exterior1st', y = 'SalePrice', data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,5)

sns.boxplot(x = 'Exterior2nd', y = 'SalePrice', data = house)
plt.xticks(rotation=90)

plt.show()

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'Utilities',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'Street', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'Neighborhood', data = house)
plt.xticks(rotation=90)
plt.subplot(2,3,4)

sns.countplot(x = 'Condition1', data = house)
plt.subplot(2,3,5)

sns.countplot(x = 'Condition2',  data = house)
plt.subplot(2,3,6)

sns.countplot(x = 'BldgType', data = house)

plt.show()

In [ ]:
#Checking skewness for other columns
house['BldgType'].value_counts()

# Dropping columns which are highly skewed 
house.drop(['Utilities','Street','Condition1','Condition2'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,1,1)

sns.boxplot(x = 'Neighborhood', y ='SalePrice', data = house)
plt.xticks(rotation=90)
plt.subplot(2,1,2)

sns.boxplot(x = 'BldgType', y = 'SalePrice', data = house)

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'LandContour',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'LandSlope', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'LotShape', data = house)
plt.subplot(2,3,4)

sns.countplot(x = 'Electrical', data = house)
plt.subplot(2,3,5)

sns.countplot(x = 'MSZoning',  data = house)
plt.subplot(2,3,6)

sns.countplot(x = 'LotConfig', data = house)

plt.show()

In [ ]:
house['LotConfig'].value_counts()

house.drop(['LandSlope','LandContour','Electrical'],axis=1,inplace=True)

house['MSZoning'] = house['MSZoning'].replace(['RH','C (all)'],'Others')

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)

sns.countplot(x = 'BsmtQual',  data = house)
plt.subplot(2,3,2)

sns.countplot(x = 'BsmtCond', data = house)
plt.subplot(2,3,3)

sns.countplot(x = 'BsmtExposure', data = house)
plt.subplot(2,3,4)

sns.countplot(x = 'BsmtFinType1', data = house)
plt.subplot(2,3,5)

sns.countplot(x = 'BsmtFinType2',  data = house)
plt.subplot(2,3,6)

sns.countplot(x = 'PavedDrive', data = house)

plt.show()

In [ ]:
#Checking skewness for other columns
house['BsmtCond'].value_counts() 

#Dropping columns which are highly skewed
house.drop(['BsmtFinType2','PavedDrive','BsmtCond'],axis=1,inplace=True)
house.head()

In [ ]:
house.info()

In [ ]:
#feature update to have better understanding

house['YearSinceRemodel'] = 2010 - ((house['YearRemodAdd'] - house['YearBuilt']) + house['YearBuilt']) 

### Dummy Variable creation

In [ ]:
#checking all categorical columns to form dummy variables

cat_cols1 = house.select_dtypes(include=['object']) 
cat_cols1.columns

In [ ]:
# Checking for numerical columns

num_cols = house.select_dtypes(include=['int64','float64'])
num_cols.columns

In [ ]:
categorical = pd.get_dummies(cat_cols1,drop_first=True)  # Dummy variables

print(categorical.shape)

In [ ]:
categorical.head()

In [ ]:
# concat dummy variables with main dataset

house = pd.concat([house, categorical], axis=1) 
house.head()

In [ ]:
#removing columns as dummy variables already formed

house.drop(['MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'BldgType',
       'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd',
       'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1',
       'HeatingQC', 'KitchenQual', 'GarageType', 'GarageFinish',
       'SaleCondition','Id','OverallCond','MSSubClass','OverallQual'],axis=1,inplace=True)

house.head()

In [ ]:
#no need of these while making a model 

house.drop(['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],axis=1,inplace=True) 
house.head()

# Model Building and Evaluation

In [ ]:
# Putting feature variable to X

X = house.drop(['SalePrice'], axis=1)

X.head()

In [ ]:
# Putting response variable to y

y = house['SalePrice']

y.head()

In [ ]:
# Test Train creation

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.7, test_size = 0.3, random_state=100)

In [ ]:
#Using StandardScaler for train data

scaler = StandardScaler()

X_train[['LotFrontage', 'LotArea','MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces','GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal']] = scaler.fit_transform(X_train[['LotFrontage', 'LotArea','MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces','GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal']])

X_train.head()

In [ ]:
#Using StandardScaler for test data

X_test[['LotFrontage', 'LotArea','MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces','GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal']] = scaler.fit_transform(X_test[['LotFrontage', 'LotArea','MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces','GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal']])

X_test.head()

## Ridge

In [ ]:
# list of alphas to tune
params = {'alpha': [0.00004,0.0001, 0.001, 0.01, 0.1, 10,100,1000]}

ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'r2', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)

model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=100]

cv_results.head()

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('r2')
plt.title("r2 and alpha")
plt.legend(['train score', 'test score'], loc='upper left')

plt.show()

In [ ]:
#optimum value of alpha

alpha = 10
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
ridge.coef_

In [ ]:
# Ridge score of train data

ridge.score(X_train,y_train)

In [ ]:
# Ridge score of test data

ridge.score(X_test,y_test)

In [ ]:
# Ridge model parameters
model_parameters = list(sorted(ridge.coef_))
model_parameters.insert(0, ridge.intercept_)
model_parameters = [round(x, 3) for x in model_parameters]
cols = X.columns
cols = cols.insert(0, "constant")
list(zip(cols, model_parameters))

## Lasso

In [ ]:
params = {'alpha': [0.00006,0.0006, 0.0008, 0.001, 0.002, 0.004, 0.006, 0.008 ]}
lasso = Lasso()

# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'r2', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.head()

In [ ]:
# plotting mean test and train scores with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('r2')

plt.title("r2 and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
#optimum value of alpha

alpha = 0.001

lasso = Lasso(alpha=alpha)
        
lasso.fit(X_train, y_train) 

In [ ]:
lasso.coef_

In [ ]:
# Lasso score of train data

lasso.score(X_train,y_train)

In [ ]:
# Lasso score of test data

lasso.score(X_test,y_test)

In [ ]:
# lasso model parameters

model_parameters = list(sorted(lasso.coef_))
model_parameters.insert(0, lasso.intercept_)
model_parameters = [round(x, 3) for x in model_parameters]
cols = X.columns
cols = cols.insert(0, "constant")

list((zip(cols, model_parameters)))

In [ ]:
# Checking r2

reg = LinearRegression()
reg.fit(X, y)

In [ ]:
# Prediction on the basis of the model
y_pred = reg.predict(X)
y_pred

In [ ]:
r2_score(y, y_pred)

In [ ]:
### Since r2 score is 0.91 which means prediction is good

# Overall Conclusion

1. Ridge - Train : 0.90, Test : 0.87
2. Lasso - Train : 0.89, Test : 0.86

## Top 10 variables in Ridge are:

Below 10 variables are significant in predicting the price of a house using Ridge:

1. SaleCondition_Partial : 0.144
2. SaleCondition_Others : 0.104
3. SaleCondition_Normal : 0.095
4. GarageFinish_Unf : 0.093
5. GarageFinish_RFn : 0.088
6. GarageFinish_No Garage : 0.075
7. GarageType_Others : 0.058
8. GarageType_No Garage : 0.057
9. GarageType_Detchd : 0.056
10. GarageType_BuiltIn : 0.054

## Top 10 variables in Lasso are:

Below 10 variables are significant in predicting the price of a house using Lasso:

1. SaleCondition_Partial : 0.201
2. SaleCondition_Others : 0.122
3. SaleCondition_Normal : 0.103
4. GarageFinish_Unf : 0.099
5. GarageFinish_RFn : 0.085
6. GarageFinish_No Garage : 0.081
7. GarageType_Others : 0.059
8. GarageType_No Garage : 0.056
9. GarageType_Detchd : 0.052
10. GarageType_BuiltIn : 0.047

## Optimal Value of lambda

Optimal Value of lambda for Ridge : 10

Optimal Value of lambda for Lasso : 0.001